In [3]:
#ACQUIRE bussines_info

import sqlite3
import pandas as pd
import numpy as np
import re

df = sqlite3.connect('/Users/MIGUEL/Desktop/CLAB1/Ironhack-Module-1-Project---Pipelines-/data/raw/Miguel318.db')

dfBI = pd.read_sql_query("SELECT * FROM business_info", df)
    
#WRANGLING bussines_info
    #drop realTimeWorth - 100% nulls

dfBI.drop(['realTimeWorth'], axis='columns', inplace=True)

    #eliminating columns that do not add value (Unnamed:0, realTimePosition)
dfBI.drop(['Unnamed: 0'], axis='columns', inplace=True)
dfBI.drop(['realTimePosition'], axis='columns', inplace=True)

dfBI.replace({"BUSD": ' '})

    #replacement of 'na' values by 0.0 in wortChange column

dfBI.worthChange.replace('na', '0.0', regex=True, inplace=True)

    #regex - separate numerical values
dfBI.worth.replace('[BUSD]+$', '', regex=True, inplace=True)
dfBI.worthChange.replace('[millions USD]+$', '', regex=True, inplace=True)

    #replacement float values with int
dfBI['worth'] = dfBI['worth'].astype('float').astype('int')
dfBI['worthChange'] = dfBI['worthChange'].astype('float').astype('int')

    #checking the type of values in columns
print(dfBI['worthChange'].dtype)
print(dfBI['worth'].dtype)

    #Analyzing worthChange column composition:
print(dfBI['worthChange'].value_counts())
print(dfBI['worthChange'].describe())

#Once the value_counts () and describe () methods have been performed, we can see that the column is composed of 97.59% of 0 values, however we can see that they include very extreme values in minimums and maximums, and since it is a column That refers to the exchange rates I think it is convenient not to eliminate the column for futures and possible analyzes during the project.

#I rename columns to indicate the units in which the values are expressed and not to contaminate the dataset.

dfBI = dfBI.rename(columns={'worth':'worth BUSD', 'worthChange':' worthChange millions USD'})

    #Make sure there are no duplicate items
dfBI = dfBI.drop_duplicates()
dfBI

#Exporting the clean dataset to the data/processed folder

dfBI.to_csv('/Users/MIGUEL/Desktop/CLAB1/Ironhack-Module-1-Project---Pipelines-/data/processed/dfBussinesInfoCLEAN.csv', index=False)




int64
int64
 0      2155
 3         5
 1         5
-3         4
-1         4
-2         3
-4         2
 4         2
 6         2
-8         2
-5         2
-18        2
-546       2
 37        1
 530       1
 2         1
 10        1
-20        1
 118       1
-672       1
-31        1
-71        1
-67        1
 5         1
 9         1
 15        1
-17        1
-19        1
 23        1
-29        1
 307       1
Name: worthChange, dtype: int64
count    2208.000000
mean       -0.459239
std        25.651478
min      -672.000000
25%         0.000000
50%         0.000000
75%         0.000000
max       530.000000
Name: worthChange, dtype: float64
